<a href="https://colab.research.google.com/github/kumar045/Assignment-For-Filed/blob/main/Indic_Gemma_7B_Lora_Inference_Unsloth_(Public).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/Telugu-LLM-Labs


Blog Post: https://ravidesetty.medium.com/introducing-indic-gemma-7b-2b-instruction-tuned-model-on-9-indian-languages-navarasa-86bc81b4a282

For even better quality output run the code on A100 with non-quantized model.

So basically change load_in_4bit = False and use the second !pip install unsloth[colab-ampere] line below if you have A100 available.

In [ ]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
# !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git". # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN_TELUGU_LLM_Labs') #Replace with your own token that has accepted Google Gemma License


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Telugu-LLM-Labs/Indic-gemma-7b-finetuned-sft-Navarasa",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:71: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

Unsloth: You passed in `google/gemma-7b` and `load_in_4bit = True`.
We shall load `unsloth/gemma-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/800M [00:00<?, ?B/s]

Unsloth 2024.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""



In [ ]:
inst = "लिंक्डइन पर कैसे बढ़ें? 5 प्वाइंट में बताएं"
input = ""

inputs = tokenizer(
[
    alpaca_prompt.format(
        inst, # instruction
        input, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# since we use packing = True it starts generating another similar sample starting with <bos>. So we are using eos_token_id = tokenizer.bos_token_id
outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

<bos>
### Instruction:
लिंक्डइन पर कैसे बढ़ें? 5 प्वाइंट में बताएं

### Input:


### Response:
1. प्रोफाइल को अद्यतन करें: अपने प्रोफाइल को अद्यतन करें और अपने क<bos>Below 1000 शब्दों में अपने करियर के बारे में बताएं।

2. लिंकडइन पर सक्रिय रहें: लिंकडइन पर सक्रिय रहें और अपने प्रोफाइल को अद्यतन करते रहें।

3. लिंकडइन पर पोस्ट करें: लिंकडइन पर पोस्ट करें और अपने करियर के बारे में जानकारी साझा करें।

4. लिंकडइन पर नेटवर्किंग करें: लिंकडइन पर नेटवर्किंग करें और अपने करियर के बारे में जानकारी साझा करें।

5. लिंकडइन पर कौशल और अनुभव को अद्यतन करें: अपने कौशल और अनुभव को अद्यतन करें और अपने प्रोफाइल को अद्यतन करते रहें।<eos>


In [ ]:
inst = "నేను భారతదేశంలో నివసిస్తున్నాను. అండర్ గ్రాడ్యుయేషన్ తర్వాత మాస్టర్స్‌కి వెళ్లడం మంచిదా లేదా నేను భారతదేశంలో కొన్ని సంవత్సరాలు పని చేయాలా?"

inputs = tokenizer(
[
    alpaca_prompt.format(
        inst, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

<bos>
### Instruction:
నేను భారతదేశంలో నివసిస్తున్నాను. అండర్ గ్రాడ్యుయేషన్ తర్వాత మాస్టర్స్‌కి వెళ్లడం మంచిదా లేదా నేను భారతదేశంలో కొన్ని సంవత్సరాలు పని చేయాలా?

### Input:


### Response:
అండర్ గ్రాడ్యుయేషన్ తర్వాత మాస్టర్స్ కు వెళ్లాలా లేదా మీ దేశంలో కొన్ని సంవత్సరాలు పని చేయాలా అనే దానిపై నిర్ణయించడం కష్టం. మీ నిర్ణయం తీసుకునేటప్పుడు మీకు సహాయపడే కొన్ని అంశాలు ఇక్కడ ఉన్నాయి:

1. కెరీర్: మీరు ఏ రంగంలో కెరీర్ కొనసాగించాలనుకుంటున్నారో పరిగణించండి. కొన్ని పరిశ్రమలు మాస్టర్స్ డిగ్రీ అవసరం కావచ్చు, మరికొన్ని అండర్ గ్రాడ్యుయేషన్ తర్వాత కొన్ని సంవత్సరాలు పని చేయడం ద్వారా అనుభవం పొందడం ద్వారా ప్రారంభించవచ్చు.

2. ఖర్చు: అండర్ గ్రాడ్యుయేషన్ తర్వాత మాస్టర్స్ కు హాజరు కావడం ఖర్చుతో కూడుకున్నది. ఇది మీకు అందుబాటులో ఉందో లేదో తెలుసుకోవడానికి మీరు ట్యూషన్, జీవన ఖర్చులు మరియు ఇతర ఖర్చులను లెక్కించాలి.

3. కెరీర్ పురోగతి: మాస్టర్స్ డిగ్రీ మీ కెరీర్ అవకాశాలను ఎలా ప్రభావితం చేస్తుందో పరిగణించండి. కొన్ని పరిశ్రమలలో, మాస్టర్స్ డిగ్రీని కలిగి ఉండటం వల్ల మీరు ఎక్కువ అవకాశాలను పొందవచ్చు లేదా పురోగతికి ముం

In [ ]:
inst = "தமிழ்நாட்டின் முக்கிய நகரங்களைப் பற்றி ஒரு வாக்கியத்தை எழுதுங்கள்"

inputs = tokenizer(
[
    alpaca_prompt.format(
        inst, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

<bos>
### Instruction:
தமிழ்நாட்டின் முக்கிய நகரங்களைப் பற்றி ஒரு வாக்கியத்தை எழுதுங்கள்

### Input:


### Response:
தமிழ்நாட்டில் உள்ள முக்கிய நகரங்களில் சென்னை, கோவாய், மதுரை, திருச்சி, தூத்துக்குடி, சூரதாமுடி, சேலம், தண்டலூர், நாகபந்தபேட்டா மற்றும் பாலக்கடலை ஆகியவை அடங்கும்.<eos>


In [ ]:
inst = "ನನ್ನ ಬಳಿ 100 ರೂಪಾಯಿ ಇದೆ. ನಾನು ಚಾಕೊಲೇಟ್‌ಗೆ 20 ಖರ್ಚು ಮಾಡಿದೆ ಮತ್ತು ನಂತರ ನನ್ನ ಸ್ನೇಹಿತನಿಗೆ 10 ರೂಪಾಯಿ ನೀಡಿದೆ. ನನ್ನ ಬಳಿ ಎಷ್ಟು ರೂಪಾಯಿ ಇದೆ?"

inputs = tokenizer(
[
    alpaca_prompt.format(
        inst, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

<bos>
### Instruction:
ನನ್ನ ಬಳಿ 100 ರೂಪಾಯಿ ಇದೆ. ನಾನು ಚಾಕೊಲೇಟ್‌ಗೆ 20 ಖರ್ಚು ಮಾಡಿದೆ ಮತ್ತು ನಂತರ ನನ್ನ ಸ್ನೇಹಿತನಿಗೆ 10 ರೂಪಾಯಿ ನೀಡಿದೆ. ನನ್ನ ಬಳಿ ಎಷ್ಟು ರೂಪಾಯಿ ಇದೆ?

### Input:


### Response:
ನೀವು 20 ರೂಪಾಯಿ ಚಾಕೊಲೇಟ್‌ಗೆ ಖರ್ಚು ಮಾಡಿದ ನಂತರ, ನಿಮ್ಮ ಬಳಿ 80 ರೂಪಾಯಿ ಉಳಿದಿದೆ. ನೀವು ನಂತರ ನಿಮ್ಮ ಸ್ನೇಹಿತರಿಗೆ 10 ರೂಪಾಯಿ ನೀಡಿದ್ದರಿಂದ, ನಿಮ್ಮ ಬಳಿ 70 ರೂಪಾಯಿ ಉಳಿದಿದೆ.<eos>
